Name: Arya Sun
<br>
Github Username: fearofhelicopter
<br>
USC ID: 8111052837

## 1. Time Series Classification Part 1: Feature Creation/Extraction
An interesting task in machine learning is classification of time series. In this problem, we will classify the activities of humans based on time series obtained by a Wireless Sensor Network.

### (a) Download Data
(a) Download the AReM data from: https://archive.ics.uci.edu/ml/datasets/Activity+Recognition+system+based+on+Multisensor+data+fusion+\%28AReM\%29 . The dataset contains 7 folders that represent seven types of activities. In each folder, there are multiple files each of which represents an instant of a human performing an activity.1 Each file containis 6 time series collected from activities of the same person, which are called avg rss12, var rss12, avg rss13, var rss13, vg rss23, and ar rss23. There are 88 instances in the dataset, each of which contains 6 time series and each time series has 480 consecutive values.



In [1]:
import os
import glob
import pandas as pd
import numpy as np

import bootstrapped.bootstrap as bs
import bootstrapped.stats_functions as bs_stats

In [2]:
# home_dir = os.getcwd()
folders = ['bending1', 'bending2', 'cycling', 'lying', 'sitting', 'standing', 'walking']
dfs = dict()
for folder in folders:
    path = os.path.join("../", "data", "AReM", folder)
    files = glob.glob(os.path.join(path, "*.csv"))
    #files.sort(key = lambda x:int(x[7:x.index('.')]))
    df = pd.concat([pd.read_csv(f, skiprows = 4) for f in files], ignore_index=True)
    dfs[folder] = df
#dfs

### (b) Test and Train Data
Keep datasets 1 and 2 in folders bending1 and bending 2, as well as datasets 1, 2, and 3 in other folders as test data and other datasets as train data.

In [3]:
traindata = []
testdata = []

for folder in os.listdir(os.path.join("../", "data", "AReM")):
    if folder in ["bendingType.pdf", "sensorsPlacement.pdf"]:
        continue
    else:
        for file in os.listdir(os.path.join("../", "data", "AReM", folder)):
            if folder in ["bending1", "bending2"] and file in ["dataset1.csv","dataset2.csv"]:
                testdata.append(os.path.join("../", "data", "AReM", folder, file))
            elif folder not in ["bending1", "bending2"] and file in ["dataset1.csv","dataset2.csv","dataset3.csv"]:
                testdata.append(os.path.join("../", "data", "AReM", folder, file))
            else:
                traindata.append(os.path.join("../", "data", "AReM", folder, file))
df_train = pd.concat([pd.read_csv(f, skiprows = 4) for f in traindata], ignore_index=True) 
df_test = pd.concat([pd.read_csv(f, skiprows = 4) for f in testdata], ignore_index=True)

#print(df_train)
#print(df_test)  

### (c) Feature Extraction
Classification of time series usually needs extracting features from them. In this problem, we focus on time-domain features.

#### i. Research

what types of time-domain features are usually used in time series classification and list them (examples are minimum, maximum, mean, etc).

Statistical: minimum, maximum, mean, median, mode, variance, covariance, zero cross rate, standard deviation, quartiles (first, third, IQR), range, root mean square, distance, skewness, kurtosis.

Time-domain: correlation structure (cross & auto), parameters of ARIMA, peak-trough, distribution, entropy, stationarity, scaling properties.

#### ii. Extraction
Extract the time-domain features minimum, maximum, mean, median, standard deviation, first quartile, and third quartile for all of the 6 time series in each instance. You are free to normalize/standardize features or use them
directly. where, for example, 1st quart6, means the first quartile of the sixth time series in each of the 88 instances.

In [4]:
rowList = []
for file in traindata:
    dfT = pd.read_csv(file, skiprows = 4)
    df1 = dfT.describe().T
    df1 = df1[["min","max", "mean", "std", "50%", "25%", "75%"]]
    df1 = df1[1:]
    row = df1.values.reshape(1, -1)
    row = pd.DataFrame(row)
    rowList.append(row)
dfFeatureTrain = pd.concat(rowList)

rowList = []
for file in testdata:
    dfT = pd.read_csv(file, skiprows = 4)
    df1 = dfT.describe().T
    df1 = df1[["min","max", "mean", "std","50%", "25%", "75%"]]
    df1 = df1[1:]
    row = df1.values.reshape(1, -1)
    row = pd.DataFrame(row)
    rowList.append(row)
dfFeatureTest = pd.concat(rowList)

dfFeature = pd.concat([dfFeatureTrain, dfFeatureTest], ignore_index=True)
dfFeature.columns = ["min1","max1", "mean1", "std1", "50%1", "25%1", "75%1",
                     "min2","max2", "mean2", "std2", "50%2", "25%2", "75%2",
                     "min3","max3", "mean3", "std3", "50%3", "25%3", "75%3",
                     "min4","max4", "mean4", "std4", "50%4", "25%4", "75%4",
                     "min5","max5", "mean5", "std5", "50%5", "25%5", "75%5",
                     "min6","max6", "mean6", "std6", "50%6", "25%6", "75%6"]

dfFeature.index = np.arange(1, len(dfFeature) + 1)
dfFeature.index.name = 'Instance'

dfFeature

,min1,max1,mean1,std1,50%1,25%1,75%1,min2,max2,mean2,...,50%5,25%5,75%5,min6,max6,mean6,std6,50%6,25%6,75%6
Instance,,,,,,,,,,,,,,,,,,,,,
1,36.25,48.00,43.969125,1.618364,44.500,43.31,44.6700,0.0,1.50,0.413125,...,21.670,20.5000,23.7500,0.0,2.96,0.555313,0.487826,0.49,0.0000,0.83
2,37.00,48.00,43.454958,1.386098,43.250,42.50,45.0000,0.0,1.58,0.378083,...,23.500,22.2500,24.0000,0.0,5.26,0.679646,0.622534,0.50,0.4300,0.87
3,33.00,47.75,42.179813,3.670666,43.500,39.15,45.0000,0.0,3.00,0.696042,...,35.000,30.4575,36.3300,0.0,2.18,0.613521,0.524317,0.50,0.0000,1.00
4,33.00,45.75,41.678063,2.243490,41.750,41.33,42.7500,0.0,2.83,0.535979,...,30.000,28.4575,31.2500,0.0,1.79,0.383292,0.389164,0.43,0.0000,0.50
5,35.00,47.40,43.954500,1.558835,44.330,43.00,45.0000,0.0,1.70,0.426250,...,36.000,35.3625,36.5000,0.0,1.79,0.493292,0.513506,0.43,0.0000,0.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,24.75,48.33,44.182937,7.495615,48.000,48.00,48.0000,0.0,3.11,0.101875,...,3.330,2.0000,5.5425,0.0,3.91,0.692771,0.675781,0.50,0.3225,0.94
85,48.00,48.25,48.004167,0.032038,48.000,48.00,48.0000,0.0,0.43,0.007167,...,9.000,4.6700,10.0000,0.0,2.50,0.641229,0.388372,0.50,0.4600,0.83
86,24.25,45.00,37.177042,3.581301,36.250,34.50,40.2500,0.0,8.58,2.374208,...,20.000,17.9500,21.7500,0.0,9.34,2.921729,1.852600,2.50,1.5000,3.90


#### iii. Standard Deviation
Estimate the standard deviation of each of the time-domain features you extracted from the data. Then, use Python’s bootstrapped or any other method to build a 90% bootsrap confidence interval for the standard deviation of each feature.

In [5]:
low = []
high = []

for c in dfFeature.columns:
    bootstrap_ci = bs.bootstrap(np.array(dfFeature[c]), stat_func = bs_stats.std, alpha = 0.1)
    low.append(bootstrap_ci.lower_bound)
    high.append(bootstrap_ci.upper_bound)

dfEST = pd.DataFrame({"stdev.": dfFeature.describe().transpose()['std'], "90% CI lower": low, "90% CI upper": high})
dfEST.index.name = 'feature'

dfEST

,stdev.,90% CI lower,90% CI upper
feature,,,
min1,9.569975,8.320801,10.835212
max1,4.394362,3.466066,5.424507
mean1,5.335718,4.748020,5.912673
std1,1.772153,1.584447,1.962801
50%1,5.440054,4.864339,6.046970
25%1,6.153590,5.614450,6.697438
75%1,5.138925,4.406730,5.927911
min2,0.000000,0.000000,0.000000
max2,5.062729,4.687222,5.471442


#### iv. Select Features
Use your judgement to select the three most important time-domain features (one option may be min, mean, and max).

Mean: central tendency and skewness

Standard Deviation: dispersion and variability

Median: average value without outliers

## 2. ISLR 3.7.4

### (a) Linear Train

The training RSS for the cubic regression is lower because it overfits with more flexibility/degrees of freedom and is closer to the data with a larger irreducible error. 

### (b) Linear Test

The testing RSS for the linear regression is lower since the true relationship is linear (lower bias and variance), and the overfit from training has more errors.

### (c) Not Linear Train

The training RSS for the cubic regression is lower due to the higher flexibility, resulting in reduced training errors. 

### (d) Not Linear Testing

There is not enough information to tell because the true relationship between X and Y is unknown. If it is far from linear, the testing RSS for the cubic regression is lower. If it is close to linear, the testing RSS for the linear regression is lower.